In [ ]:
import numpy as np
import pandas as pd
import fastf1


# -----------------------------
# Setup
# -----------------------------
def setup_fastf1_cache(cache_dir: str = "./fantasy_cache"):
    """
    Enable local caching. Massive speed-up after first run.
    """
    fastf1.Cache.enable_cache(cache_dir)


# -----------------------------
# Helpers
# -----------------------------
def _to_seconds(x) -> float:
    """
    Convert pandas Timedelta / datetime-like to seconds (float).
    Returns np.nan if missing.
    """
    if pd.isna(x):
        return np.nan
    try:
        return x.total_seconds()
    except Exception:
        return np.nan


def _safe_event_name(event_row) -> str:
    # FastF1 event row has 'EventName' or 'EventName' depending on version
    for k in ["EventName", "EventName"]:
        if k in event_row and pd.notna(event_row[k]):
            return str(event_row[k])
    # fallback
    return str(event_row.get("OfficialEventName", "UnknownEvent"))


def get_events(year: int) -> pd.DataFrame:
    """
    Returns event schedule as a DataFrame.
    """
    cal = fastf1.get_event_schedule(year)
    # Keep only race weekends (some versions already only include race weekends)
    return cal


def load_session(year: int, round_no: int, session_code: str):
    """
    session_code: 'Q' for qualifying, 'R' for race
    """
    s = fastf1.get_session(year, round_no, session_code)
    s.load()
    return s


# -----------------------------
# Core metrics per event
# -----------------------------
def qualifying_metrics_for_event(year: int, round_no: int) -> pd.DataFrame:
    """
    Per-driver qualifying metrics from a single event.
    Returns one row per driver with:
    - grid position
    - best q time (as seconds) if available
    - reached Q3 (boolean)
    """
    q = load_session(year, round_no, "Q")
    res = q.results.copy()

    # Typical columns in results: 'Abbreviation', 'TeamName', 'Position', 'Q1', 'Q2', 'Q3'
    # Some weekends may have sprint quali; we stick to 'Q' session.
    out = pd.DataFrame({
        "year": year,
        "round": round_no,
        "driver": res.get("Abbreviation", res.get("DriverNumber")).astype(str),
        "team": res.get("TeamName", "").astype(str),
        "q_position": pd.to_numeric(res.get("Position", np.nan), errors="coerce"),
        "q1_s": res.get("Q1", np.nan).apply(_to_seconds),
        "q2_s": res.get("Q2", np.nan).apply(_to_seconds),
        "q3_s": res.get("Q3", np.nan).apply(_to_seconds),
    })

    # Best available quali time (Q3 preferred, else Q2, else Q1)
    out["best_q_s"] = out["q3_s"].combine_first(out["q2_s"]).combine_first(out["q1_s"])
    out["reached_q3"] = out["q3_s"].notna()

    return out


def race_metrics_for_event(year: int, round_no: int) -> pd.DataFrame:
    """
    Per-driver race metrics from a single event.
    Returns one row per driver with:
    - start grid, finish position (where available)
    - positions gained
    - classified (finished / classified)
    - status (string if available)
    """
    r = load_session(year, round_no, "R")
    res = r.results.copy()

    # Columns often: 'GridPosition', 'Position', 'Status', 'Abbreviation', 'TeamName'
    out = pd.DataFrame({
        "year": year,
        "round": round_no,
        "driver": res.get("Abbreviation", res.get("DriverNumber")).astype(str),
        "team": res.get("TeamName", "").astype(str),
        "grid": pd.to_numeric(res.get("GridPosition", np.nan), errors="coerce"),
        "finish": pd.to_numeric(res.get("Position", np.nan), errors="coerce"),
        "status": res.get("Status", "").astype(str),
    })

    # "classified" heuristic: finish position present AND not obviously DNF/DNS/DSQ
    # FastF1 Status sometimes contains 'Finished', '+1 Lap', 'Accident', 'Retired', 'Disqualified', etc.
    status_lower = out["status"].str.lower()
    out["classified"] = out["finish"].notna() & ~status_lower.str.contains("disqual|dsq|dns", na=False)

    # positions gained: grid - finish (bigger positive = gained places)
    out["pos_gained"] = out["grid"] - out["finish"]

    # DNF-ish: not classified or explicit retired/accident/mechanical
    out["dnf_like"] = ~out["classified"] | status_lower.str.contains("retired|accident|collision|engine|gearbox|hydraul|electrical|puncture", na=False)

    return out


# -----------------------------
# Team internal gap (Qualifying)
# -----------------------------
def teammate_quali_gap(qual_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes per-event teammate gap on best qualifying time:
    gap_s = driver's best_q_s - teammate best_q_s (within same team & event)
    Negative = faster than teammate.
    """
    df = qual_df.copy()

    # For each (year, round, team), find the best_q_s for both drivers and map teammate best.
    # We assume two drivers per team; if more due to substitutions, this still works via "best of others".
    df["team_best_q_s"] = df.groupby(["year", "round", "team"])["best_q_s"].transform("min")

    # Teammate best: min of others in same team
    def _teammate_best(group):
        vals = group["best_q_s"].values
        out = []
        for i, v in enumerate(vals):
            others = np.delete(vals, i)
            if len(others) == 0:
                out.append(np.nan)
            else:
                out.append(np.nanmin(others))
        return pd.Series(out, index=group.index)

    df["teammate_best_q_s"] = df.groupby(["year", "round", "team"], group_keys=False).apply(_teammate_best)
    df["teammate_gap_s"] = df["best_q_s"] - df["teammate_best_q_s"]

    return df[["year", "round", "driver", "team", "best_q_s", "reached_q3", "q_position", "teammate_gap_s"]]


# -----------------------------
# Aggregate across first N races
# -----------------------------
def build_fantasy_features(year: int, n_rounds: int = 6) -> dict:
    """
    Pulls data for first n_rounds of year and returns:
    - driver_features: aggregated per-driver
    - constructor_features: aggregated per-team
    - per_event_driver: event-level joined table (Q + R)
    """
    events = get_events(year)
    # Some schedules include pre-season/testing; keep the first n_rounds race weekends by RoundNumber
    events = events.sort_values("RoundNumber")
    rounds = [int(r) for r in events["RoundNumber"].dropna().unique()][:n_rounds]

    qual_all = []
    race_all = []

    for rnd in rounds:
        qual_all.append(qualifying_metrics_for_event(year, rnd))
        race_all.append(race_metrics_for_event(year, rnd))

    qual_df = pd.concat(qual_all, ignore_index=True)
    race_df = pd.concat(race_all, ignore_index=True)

    # teammate gap based on quali
    gap_df = teammate_quali_gap(qual_df)

    # Join per-event driver table
    per_event_driver = (
        gap_df
        .merge(race_df[["year", "round", "driver", "grid", "finish", "status", "classified", "pos_gained", "dnf_like"]],
               on=["year", "round", "driver"], how="left")
    )

    # -------------------------
    # Driver-level aggregation
    # -------------------------
    driver_features = per_event_driver.groupby(["driver", "team"], as_index=False).agg(
        races=("round", "nunique"),
        q3_rate=("reached_q3", "mean"),
        median_q_pos=("q_position", "median"),
        median_pos_gained=("pos_gained", "median"),
        mean_pos_gained=("pos_gained", "mean"),
        dnf_rate=("dnf_like", "mean"),
        median_teammate_gap_s=("teammate_gap_s", "median"),
        mean_teammate_gap_s=("teammate_gap_s", "mean"),
    )

    # -------------------------
    # Constructor-level aggregation
    # -------------------------
    # "both cars classified" per round & team
    team_round = per_event_driver.groupby(["team", "round"], as_index=False).agg(
        cars=("driver", "nunique"),
        classified_count=("classified", "sum"),
        dnf_like_count=("dnf_like", "sum"),
        points_proxies=("finish", lambda s: np.nanmean(s))  # not points; just a proxy
    )
    team_round["both_classified"] = (team_round["cars"] >= 2) & (team_round["classified_count"] >= 2)

    constructor_features = team_round.groupby("team", as_index=False).agg(
        rounds=("round", "nunique"),
        both_classified_rate=("both_classified", "mean"),
        avg_dnf_like_per_round=("dnf_like_count", "mean"),
        avg_finish_proxy=("points_proxies", "mean"),
    )

    return {
        "driver_features": driver_features.sort_values(["dnf_rate", "median_q_pos"]),
        "constructor_features": constructor_features.sort_values(["avg_dnf_like_per_round"], ascending=True),
        "per_event_driver": per_event_driver.sort_values(["round", "team", "driver"])
    }


# -----------------------------
# Simple "Fantasy Value" Scores (no prices needed)
# -----------------------------
def add_value_scores(driver_features: pd.DataFrame, constructor_features: pd.DataFrame):
    """
    Creates heuristic scores. Lower = better.
    You can later merge your fantasy prices and compute "value per cost".
    """
    d = driver_features.copy()

    # Normalize into comparable scales
    # (robust enough for small samples; in interviews say: "simple z-scoring for comparability")
    def z(x):
        x = x.astype(float)
        mu, sd = np.nanmean(x), np.nanstd(x)
        if sd == 0 or np.isnan(sd):
            return (x - mu)
        return (x - mu) / sd

    # Driver score: prioritize quali + reliability + teammate dominance + pos gains
    d["score_driver"] = (
        0.40 * z(d["median_q_pos"]) +                 # lower better
        0.30 * z(d["dnf_rate"]) +                     # lower better
        0.20 * z(d["median_teammate_gap_s"]) -        # negative gap = good, so higher gap is bad
        0.10 * z(d["median_pos_gained"])              # higher pos gained is good -> subtract
    )

    c = constructor_features.copy()
    c["score_constructor"] = (
        0.60 * z(c["avg_dnf_like_per_round"]) -       # lower better
        0.40 * z(c["both_classified_rate"])           # higher better -> subtract
    )

    return d.sort_values("score_driver"), c.sort_values("score_constructor")


# -----------------------------
# Example usage
# -----------------------------
if __name__ == "__main__":
    setup_fastf1_cache("../fantasy_cache")

    YEAR = 2025      # change to 2026 when season data exists

    data = build_fantasy_features(YEAR, n_rounds=N_ROUNDS)

    drivers = data["driver_features"]
    constructors = data["constructor_features"]

    drivers_scored, constructors_scored = add_value_scores(drivers, constructors)

    print("\n=== Driver shortlist (heuristic, no prices yet) ===")
    print(drivers_scored.head(15).to_string(index=False))

    print("\n=== Constructor shortlist (heuristic) ===")
    print(constructors_scored.head(10).to_string(index=False))


ValueError: Cannot get testing event by round number!

In [28]:
def welcome(name):
  return "Welcome, " + name
greet = welcome

greet('klaus')

a = 1
second = "mJnA dLaimenbje"
print(f"My mama {1}","dont like you")

print(second[:4:3])

My mama 1 dont like you
mA


In [102]:
def is_leap(year):
    leap = False
    
    # Write your logic here
    # devidable by 4 == leap
    if year % 4 == 0 and year % 100 == 0 and year % 400 == 0:
        print("reached")
        if year % 100 == 0:
            leap = False
        if year % 100 == 0 and year % 400 == 0:
            leap = True
    return leap

year = 1996
print(is_leap(year))

False


In [103]:
i = -12312

abs(i)

12312

In [48]:
sum = [i-1 for i in arr]

sum

[0, 1, 2, 3, 4]